In [10]:
import pandas as pd

df = pd.read_csv('listings-summary.csv')

In [11]:
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,109,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.98209,-118.38494,Entire home/apt,115,30,2,2016-05-15,0.02,1,0
1,2708,Runyon Canyon | Beau Furn Mir Mini-Suite Fire...,3008,Chas.,City of Los Angeles,Hollywood,34.09768,-118.34602,Private room,71,30,28,2020-11-08,0.34,2,315
2,2732,Zen Life at the Beach,3041,Yoga Priestess,Other Cities,Santa Monica,34.00475,-118.48127,Private room,155,1,21,2019-12-27,0.18,2,365
3,2864,* Beautiful Master Suite/Jacuzzi Tub/*,3207,Bernadine,Other Cities,Bellflower,33.87619,-118.11397,Entire home/apt,50,30,0,NaN,NaN,1,0
4,5728,Tiny Home in Artistic Oasis near Venice and LAX,9171,Sanni,City of Los Angeles,Del Rey,33.98750,-118.43200,Private room,42,30,310,2020-04-29,2.20,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31895,48070933,clean and comfortable room to rest,387568794,Sergio,Unincorporated Areas,Rowland Heights,33.98763,-117.88529,Private room,36,1,0,NaN,NaN,3,180
31896,48070985,quiet and clean room,387568794,Sergio,Unincorporated Areas,Rowland Heights,33.98740,-117.88588,Private room,28,1,0,NaN,NaN,3,179
31897,48071306,LOWER PRICE and Deep Cleaning+Pool+Free Parkin...,106941010,Linda,City of Los Angeles,Westwood,34.06114,-118.44367,Entire home/apt,176,1,0,NaN,NaN,13,270
31898,48071488,Historical Mansion Hotel,8402388,Jessica,Other Cities,Glendora,34.15155,-117.87664,Entire home/apt,195,1,0,NaN,NaN,5,362
